In [1]:
# input - 224*224*3
#  3-3 convolution2 with 64 channel
# 3-3 convolution2 with 128 channel
# 3-3 convolution3 with 256 channel
# 3-3 convolution3 with 512 channel
# 3-3 convolution3 with 512 channel
# fc layer (4096^ , 1000) 3개
# -> 총 16개 layer (conv 13 , fc 3)

# stride 1 로 고정
# Act - ReLu
# Drop Out
# Softmax

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_3_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [3]:
# VGG

class VGG16(nn.Module):
    def __init__(self,base_dim=64, num_classes=10): #64, 10 으로 설정
        super(VGG16, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim), #RGB3 -> 64
            conv_2_block(base_dim,2*base_dim), #64 -> 128
            conv_3_block(2*base_dim,4*base_dim), #128 -> 256
            conv_3_block(4*base_dim,8*base_dim), #256 -> 512
            conv_3_block(8*base_dim,8*base_dim), #512 -> 512
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim*1*1,4096), #conv layer filter 수 * Base dim * feature map size
            nn.ReLU(),
            nn.Linear(4096,4096),
            nn.ReLU(), #ReLu activation
            nn.Dropout(),  #DropOut on
            nn.Linear(4096,1000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1000,num_classes),
        )
    def forward(self, x):
        x = self.feature(x)
        #print(x.shape)
        x = x.view(x.size(0),-1)
        #print(x.shape)
        x = self.fc_layer(x)
        probas = F.softmax(x,dim=1)
        return probas

In [4]:
# Dataset load
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# hyperparams
batch_size = 10
learning_rate = 0.02
num_epoch = 10

# gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

# Data - Tensor 변환 , 정규화
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)


# Transform 적용
cifar10_train = datasets.CIFAR10(root='./Data/',train=True,transform=transform,target_transform=None,download=True)
cifar10_test = datasets.CIFAR10(root="./Data/", train=False, transform=transform, target_transform=None, download=True)

# Train,Test Load
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=batch_size)

cuda:0


100%|██████████| 170498071/170498071 [00:04<00:00, 39889598.72it/s]


Extracting ./Data/cifar-10-python.tar.gz to ./Data/
Files already downloaded and verified


In [5]:
# Train
model = VGG16().to(device) # GPU 할당
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #Adam Optimizer

In [6]:
#loss 저장
loss_arr = []

for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad() #optimizer의 gradient를 초기화
        output = model.forward(x) #VGG class 의 forward prop
        # _, output = torch.max(output, 1)
        #print(output.shape)
        #print(y_.shape)
        loss = loss_func(output,y_)
        loss.backward() #CrossEntropy loss function 에 대한 back prop
        optimizer.step() #Optimizer 갱신

    if i%10 ==0:
        print(f'epcoh {i} loss : ',loss)
        loss_arr.append(loss.cpu().detach().numpy()) #detach tensor를 gradient 연산에서 분리

# 모델 저장
#torch.save(model.state_dict(), "./train_model/VGG16_trial.pth")

epcoh 0 loss :  tensor(2.4612, device='cuda:0', grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 